In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:

!pip install -q tavily-python langchain_community langchain_openai

You should consider upgrading via the 'C:\Users\sandeep.nataraja\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path = ".env")

True

In [4]:
from langchain_openai import AzureChatOpenAI
GPT_DELPOYMENT_NAME = os.getenv("DEPLOYMENT_NAME")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_APIKEY")
os.environ["OPENAI_API_VERSION"] = "2024-05-01-preview"
llm = AzureChatOpenAI(model=GPT_DELPOYMENT_NAME,azure_endpoint = os.getenv("OPENAI_ENDPOINT"))

In [5]:
llm.invoke("What's date today?")

AIMessage(content="I'm sorry, I am a language model AI and I do not have real-time capabilities to provide the current date. Please check your device or computer for the most up-to-date information.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 12, 'total_tokens': 50, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_808245b034', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filt

here is the response from normal llm call, where it's not able to answer realtime data, or it halucinate with wrong answers

So, will explore agentic approach with additional tools to feed in realtime data's and will check it's abilities

Will add two tools for now, tavily api as search tool and llm's math for calculations

In [6]:
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain.tools.retriever import create_retriever_tool
os.environ['TAVILY_API_KEY'] = os.getenv("TAVILY_KEY")

In [7]:
from langchain.tools import Tool
from langchain.chains import LLMMathChain

In [8]:
calc_chain = LLMMathChain.from_llm(llm)
calc_tool = Tool.from_function(
    name = "Calculator",
    func = calc_chain.run,
    description = "This tool should be used to answer mathematical questions where calculation might be needed"
)
search_tool = Tool.from_function(
    name = "Searchweb",
    func = create_retriever_tool(name = "Searchweb",retriever = TavilySearchAPIRetriever(),description = "This tool should be used when answer needs to be generated from current time events happening"),
    description = "This tool should be used when answer needs to be generated from current time events happening"
)

In [9]:
tools = [calc_tool,search_tool]

In [10]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key = "chat_history")

C:\Users\sandeep.nataraja\AppData\Local\Temp\ipykernel_33520\1109017631.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key = "chat_history")


In [11]:
from langchain.agents import AgentType
from langchain.agents import initialize_agent

In [12]:
agent = initialize_agent(tools = tools,llm = llm,verbose= True, memory = memory)

C:\Users\sandeep.nataraja\AppData\Local\Temp\ipykernel_33520\4149717365.py:1: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(tools = tools,llm = llm,verbose= True, memory = memory)


In [13]:
agent.run("Whats' date today?")

C:\Users\sandeep.nataraja\AppData\Local\Temp\ipykernel_33520\836767421.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Whats' date today?")




> Entering new AgentExecutor chain...
I should use the Searchweb tool to find the current date.
Action: Searchweb
Action Input: "current date"
Observation: Paris. Sat, Nov 16. 09:41 PM. Los Angeles Chicago Toronto Mexico Sao Paulo Berlin Moscow Dubai Mumbai Singapore Sydney. Check the current date and time for any location in the world. Find date, time, timezone and more for over 200 countries.

Today's Date and Time - Date and Time Tools Today's Date & Time Home Days Calculator Weeks Calculator Time Calculator Calendar Home Days Calculator Weeks Calculator Time Calculator Calendar Today's Date and Time Days Calculator Weeks Calculator What is today's date? Full Date: Date Writing Standards There are two main standards for writing dates: American English: Month-Day-Year (e.g., August 31, 2024) British English: Day-Month-Year (e.g., 31 August 2024) Calculate the number of days between two dates or add/subtract days from a date. Week Calculator Calculate the number of weeks between dat

"Today's date is Monday, November 18, 2024."

now that agent used search tool to get todays day(20/11/2024), but after analysing all search results it still failed and gave 18/11/2024.


Will check adding a function tool to return todays date

In [14]:
def date_func(*args,**kwargs):
    import datetime
    return datetime.date.today()
date_tool = Tool.from_function(
    name = "date_tool",
    func = date_func,
    description = "This tool should be used when you need to get current date, when date or day is crucial to answer"
)

In [15]:
tools = [calc_tool,date_tool,search_tool]
agent = initialize_agent(tools = tools,llm = llm,agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,verbose= True, memory = memory)

In [16]:
agent.run("Whats' date today?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: date_tool
Action Input: current date
Observation: 2024-11-20
Thought:Do I need to use a tool? No
AI: Today's date is November 20, 2024.

> Finished chain.


"Today's date is November 20, 2024."

now using date tool, it's able to return right date

In [17]:
agent.run("Whats the top gainer in nasdaq yestrday?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Searchweb
Action Input: Top gainer in NASDAQ yesterday
Observation: 6.132M. 51.599B. 32.92. Show 25 rows. Advertisement. See a list of Day Gainers - NASDAQ 100 using the Yahoo Finance screener. Create your own screens with over 150 different screening criteria.

299,485. 5.95B. 32.87. Show 25 rows. See the list of the top gaining stocks today, including share price change and percentage, trading volume, intraday highs and lows, and day charts.

Nasdaq 100 Gainers Ark ETFs Top Gainers & Losers S&P 500 Gainers Nasdaq 100 Gainers Dow Jones Gainers S&P 500 Losers Nasdaq 100 Losers Dow Jones Losers Companies Historical Returns Component Performance Year-to-date Return Dividend Yield Weight Analysis Return Components Market Capitalization Drawdown Companies Historical Returns Component Performance Year-to-date Return Dividend Yield Weight Analysis QQQ Ratio Companies Historical Returns Component Performance

'The top gainer in NASDAQ yesterday was Advanced Micro Devices Inc (AMD) with a gain of 2.99%.'

now for the stock detail, it used search tool and returned result, now without giving much conext of above question will ask followup along with math on the same, so it should give right answer invoking multiple tools

In [18]:
agent.run("Whats' average change in percentage for those top two?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: (2.99 + 1.88) / 2
Observation: Answer: 2.435
Thought:Do I need to use a tool? No
AI: The average change in percentage for the top two gainers in NASDAQ yesterday was 2.435%.

> Finished chain.


'The average change in percentage for the top two gainers in NASDAQ yesterday was 2.435%.'

It was still able to keep context of earlier query as well answer from the same, so no call happened to search api, but just math calculation on earlier computed results

In [19]:
agent.run("What was the net percentage change in value of Nvdia in the past 3 days?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Searchweb
Action Input: Net percentage change in value of Nvidia in the past 3 days
Observation: Nvidia sees over $500 billion wiped off value after shares slide—the biggest 3-day value loss for any company in history. BY Christiaan Hetzner. June 25, 2024, 5:12 AM PDT. Nvidia founder and

The closing share price for NVIDIA (NVDA) stock was $141.98 for Friday, November 15 2024, down 3.26% from the previous day. NVDA is up 4.61% over the past 30 days and up 190.42% for the past 12 months.

Market Cap. : $3.434T. The current market cap or net worth for NVIDIA (NVDA) stock is $3.434T as of Monday, November 18 2024. It's decreased by 2.58% over the past 30 days. NVDA has improved its market cap by 175.82% over the past 12 months.

587.84. +2.52%. Discover historical prices for NVDA stock on Yahoo Finance. View daily, weekly or monthly format back to when NVIDIA Corporation stock was issued.

Get free histo

'The net percentage change in value of Nvidia in the past 3 days was a significant loss of over $500 billion.'